In [9]:

# BOILER PLATE, MUST BE RUN ON SUBMIT NODE
%load_ext autoreload
%autoreload 2
import socket
print("Running on host: ", socket.gethostname())

import sys 
lib_path = '/fsx_0/user/tranx/experiments'
if lib_path not in sys.path:
    sys.path.append(lib_path)
#=================================================
    
import plotly.express as px
import plotly.graph_objects as go
import re
import time

import json
import glob
import pandas as pd
import os

from lib import eval_helper
from lib import utils
from lib import slurm
from lib.slurm import run_sbatch_job


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Running on host:  submit-1


In [6]:
params = {
    "eval_plan": "Llama31_336px",
    "json_config": "/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_vqa.json",
    "checkpoint_dir": "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch",
    "benchmark_name": "vqa",
    "checkpoint_id": "2000"
}

job = run_job(
    sbatch_base_script="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    sbatch_overwrite={
        "job-name": "eval_" + params["benchmark_name"]
    },
    positional_env_vars=list(params.values())
)

job

sbatch --parsable --job-name=eval_vqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_vqa.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch vqa 2000


9675

# Eval plan

## Llama31

In [42]:
# Llama31
EVAL_PLAN = "Llama31_336px"
BASE_SCRIPT = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
CONFIG_DIR_LLAMA31 = "/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31"
CHECKPOINT_DIR_LLAMA31 = "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch"

all_benchmarks = "mmmu docvqa mathvista ai2d chartqa vqa textvqa infographics_w_ocr infographics mmbench"
checkpoint_start = 2000
checkpoint_end = 8000
checkpoint_int = 1000

job_dict = {}
for benchmark in all_benchmarks.split():
    job_dict[benchmark] = {}

    for chk in range(checkpoint_start, checkpoint_end + checkpoint_int, checkpoint_int):
        params = {
            "eval_plan": EVAL_PLAN,
            "json_config": f"{CONFIG_DIR_LLAMA31}/eval_{benchmark}.json",
            "checkpoint_dir": CHECKPOINT_DIR_LLAMA31,
            "benchmark_name": benchmark,
            "checkpoint_id": str(chk)
        }

        assert os.path.exists(params["json_config"])
        assert os.path.exists(f"{params['checkpoint_dir']}/checkpoint-{chk}")

        job_id = run_sbatch_job(
            sbatch_base_script=BASE_SCRIPT,
            sbatch_overwrite={
                "job-name": f"eval_{benchmark}"
            },
            positional_env_vars=list(params.values())
        )

        job_dict[benchmark][chk] = int(job_id)

with open(f'job_dict_{EVAL_PLAN}.json', 'w') as f:
    json.dump(job_dict, f, indent=4)

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 2000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 3000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 4000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json 

In [113]:
# Llama31
EVAL_PLAN = "Llama31_336px_8500_9500"
checkpoints = [8500, 9000, 9500]

job_dict_json = f'job_dict_{EVAL_PLAN}.json'
assert not os.path.exists(job_dict_json)

BASE_SCRIPT = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
CONFIG_DIR_LLAMA31 = "/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31"
CHECKPOINT_DIR_LLAMA31 = "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch"

all_benchmarks = "mmmu docvqa mathvista ai2d chartqa vqa textvqa infographics_w_ocr infographics mmbench"

job_dict = {}
for benchmark in all_benchmarks.split():
    job_dict[benchmark] = {}

    for chk in checkpoints:
        params = {
            "eval_plan": EVAL_PLAN,
            "json_config": f"{CONFIG_DIR_LLAMA31}/eval_{benchmark}.json",
            "checkpoint_dir": CHECKPOINT_DIR_LLAMA31,
            "benchmark_name": benchmark,
            "checkpoint_id": str(chk)
        }

        assert os.path.exists(params["json_config"])
        assert os.path.exists(f"{params['checkpoint_dir']}/checkpoint-{chk}")

        job_id = run_job(
            sbatch_base_script=BASE_SCRIPT,
            sbatch_overwrite={
                "job-name": f"eval_{benchmark}"
            },
            positional_env_vars=list(params.values())
        )

        job_dict[benchmark][chk] = int(job_id)

with open(job_dict_json, 'w') as f:
    json.dump(job_dict, f, indent=4)

print(f"Saved job_dict to {job_dict_json}")

AssertionError: 

In [73]:
job_dict

{'mmmu': {2000: 9759,
  3000: 9760,
  4000: 9761,
  5000: 9762,
  6000: 9763,
  7000: 9764,
  8000: 9765},
 'docvqa': {2000: 9766,
  3000: 9767,
  4000: 9768,
  5000: 9769,
  6000: 9770,
  7000: 9771,
  8000: 9772},
 'mathvista': {2000: 9773,
  3000: 9774,
  4000: 9775,
  5000: 9776,
  6000: 9777,
  7000: 9778,
  8000: 9779},
 'ai2d': {2000: 9780,
  3000: 9781,
  4000: 9782,
  5000: 9783,
  6000: 9784,
  7000: 9785,
  8000: 9786},
 'chartqa': {2000: 9787,
  3000: 9788,
  4000: 9789,
  5000: 9790,
  6000: 9791,
  7000: 9792,
  8000: 9793},
 'vqa': {2000: 9794,
  3000: 9795,
  4000: 9796,
  5000: 9797,
  6000: 9798,
  7000: 9799,
  8000: 9800},
 'textvqa': {2000: 9801,
  3000: 9802,
  4000: 9803,
  5000: 9804,
  6000: 9805,
  7000: 9806,
  8000: 9807},
 'infographics_w_ocr': {2000: 9808,
  3000: 9809,
  4000: 9810,
  5000: 9811,
  6000: 9812,
  7000: 9813,
  8000: 9814},
 'infographics': {2000: 9815,
  3000: 9816,
  4000: 9817,
  5000: 9818,
  6000: 9819,
  7000: 9820,
  8000: 9821},
 'm

## Llama31 - 10000

In [3]:
job_dict_lm31_10k = eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    eval_plan="Llama31_10k",
    eval_config_dir="/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31",
    checkpoint_dir="/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch",
    checkpoints=[10000],
    # benchmarks=["mathvista"]
)

job_dict_lm31_10k

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_10k /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 10000
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_10k /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_docvqa.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch docvqa 10000
sbatch --parsable --job-name=eval_mathvista /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_10k /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mathvista.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mathvista 10000
sbatch --parsable --job-name=eval_ai2d /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_10k /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31

{'mmmu': {10000: 10008},
 'docvqa': {10000: 10009},
 'mathvista': {10000: 10010},
 'ai2d': {10000: 10011},
 'chartqa': {10000: 10012},
 'vqa': {10000: 10013},
 'textvqa': {10000: 10014},
 'infographics_w_ocr': {10000: 10015},
 'infographics': {10000: 10016},
 'mmbench': {10000: 10017}}

## Llama31 - fb

In [6]:
job_dict_fb_lm31 = eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    eval_plan="FBL_Llama31",
    eval_config_dir="/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31",
    checkpoint_dir="/fsx_0/checkpoints/tranx/fb_llama3.1",
    checkpoints=[8000, 10000],
    # benchmarks=["mathvista"]
)

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh FBL_Llama31 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/fb_llama3.1 mmmu 8000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh FBL_Llama31 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/fb_llama3.1 mmmu 10000
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh FBL_Llama31 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_docvqa.json /fsx_0/checkpoints/tranx/fb_llama3.1 docvqa 8000
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh FBL_Llama31 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_docvqa.json /fsx_0/checkpoints/tranx/fb_llama3.1 docvqa 10000
sbatch --parsable --job-name=eval_mathvista /fsx_0/user/tranx/experime

## MH19

In [1]:
checkpoints = [10500 + i*1000 for i in range(9)]
checkpoints

[10500, 11500, 12500, 13500, 14500, 15500, 16500, 17500, 18500]

In [3]:
job_dict_mh19 = eval_helper.run_eval_plan(
    eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    eval_plan="MH19_10500_18500",
    eval_config_dir="/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval",
    checkpoint_dir="/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/MH19_336px_128nodes_bz32_resume",
    checkpoints=[10500 + i*1000 for i in range(9)],
    # benchmarks=["mathvista"]
)

RuntimeError: Found existing job_dict: eval/logs/job_dict_MH19_10500_18500.json

In [12]:
df = eval_helper.get_eval_scores(job_dict_mh19)

Got result for mmmu - 10500
Got result for mmmu - 11500
Got result for mmmu - 12500
Got result for mmmu - 13500
Got result for mmmu - 14500
Got result for mmmu - 15500
Got result for mmmu - 16500
Got result for mmmu - 17500
Got result for mmmu - 18500
Got result for docvqa - 10500
Got result for docvqa - 11500
Got result for docvqa - 12500
Got result for docvqa - 13500
Got result for docvqa - 14500
Got result for docvqa - 15500
Got result for docvqa - 16500
Got result for docvqa - 17500
Got result for docvqa - 18500
Got result for mathvista - 10500
Got result for mathvista - 11500
Got result for mathvista - 12500
Got result for mathvista - 13500
Got result for mathvista - 14500
Got result for mathvista - 15500
Got result for mathvista - 16500
Got result for mathvista - 17500
Got result for mathvista - 18500
Got result for ai2d - 10500
Got result for ai2d - 11500
Got result for ai2d - 12500
Got result for ai2d - 13500
Got result for ai2d - 14500
Got result for ai2d - 15500
Got result fo

In [13]:
df.loc[17500]

mmmu/accuracy                                    0.5878
mmmu/mllm_eval_accuracy                          0.5889
docvqa/anls_total_score                          0.6201
docvqa/mllm_evaluation_anls_score                0.6216
mathvista/accuracy                                0.377
ai2d/accuracy                                    0.7704
chartqa/accuracy                                 0.4914
vqa/accuracy                                     0.7035
vqa/mllm_evaluation_accuracy                     0.7256
textvqa/accuracy                                 66.898
textvqa/mllm_eval_accuracy                       71.578
infographics_w_ocr/anls_total_score              0.6256
infographics_w_ocr/mllm_evaluation_anls_score    0.5712
infographics/anls_total_score                    0.4798
infographics/mllm_evaluation_anls_score          0.4177
mmbench/overall                                  0.7422
Name: 17500, dtype: object

# Parse results

In [1]:
# job_dict_file = 'job_dict_Llama31_336px.json'
job_dict_file = 'job_dict_Llama31_336px_8500_9500.json'

with open(job_dict_file, 'r') as f:
    job_dict = json.load(f)

df = eval_helper.get_eval_scores(job_dict)
df

NameError: name 'json' is not defined

In [4]:
job_dict_file = 'job_dict_FBL_Llama31.json'
with open(job_dict_file, 'r') as f:
    job_dict = json.load(f)

df2 = eval_helper.get_eval_scores(job_dict)
df2

Got result for mmmu - 8000
Got result for mmmu - 10000
Got result for docvqa - 8000
Got result for docvqa - 10000
Got result for mathvista - 8000
Got result for mathvista - 10000
Got result for ai2d - 8000
Got result for ai2d - 10000
Got result for chartqa - 8000
Got result for chartqa - 10000
Got result for vqa - 8000
Got result for vqa - 10000
Got result for textvqa - 8000
Got result for textvqa - 10000
Got result for infographics_w_ocr - 8000
Got result for infographics_w_ocr - 10000
Got result for infographics - 8000
Got result for infographics - 10000
Got result for mmbench - 8000
Got result for mmbench - 10000


,mmmu/accuracy,mmmu/mllm_eval_accuracy,docvqa/anls_total_score,docvqa/mllm_evaluation_anls_score,mathvista/accuracy,ai2d/accuracy,chartqa/accuracy,vqa/accuracy,vqa/mllm_evaluation_accuracy,textvqa/accuracy,textvqa/mllm_eval_accuracy,infographics_w_ocr/anls_total_score,infographics_w_ocr/mllm_evaluation_anls_score,infographics/anls_total_score,infographics/mllm_evaluation_anls_score,mmbench/overall
8000,0.5067,0.5156,0.5386,0.5411,0.344,0.7121,0.4746,0.6831,0.7137,59.028,65.38,0.5902,0.536,0.4384,0.3746,0.6635
10000,0.4744,0.4822,0.54,0.5432,0.348,0.6953,0.4578,0.6931,0.7224,59.414,65.708,0.592,0.5354,0.4472,0.3785,0.6772


# Stage 2

In [10]:
!ls /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28

checkpoint-1000  checkpoint-1400  checkpoint-400  checkpoint-800  tensorboard
checkpoint-1200  checkpoint-200   checkpoint-600  runs


In [7]:
checkpoints = list(range(200, 1000 + 200, 200))
# len(checkpoints) * len(eval_helper.ALL_BENCHMARKS)
# checkpoints = [4800]
checkpoints

[200, 400, 600, 800, 1000]

In [13]:
EVAL_SBATCH = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
EVAL_CONFIG_DIR = "/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval"

## Exp 28

In [11]:
checkpoints = [1200, 1400]
for c in checkpoints:
    plan_dict = eval_helper.run_eval_plan(
        eval_base_sbatch=EVAL_SBATCH,
        eval_plan=f"stage2_MH19_336px_128nodes_exp28_{c}",
        eval_config_dir=EVAL_CONFIG_DIR,
        checkpoint_dir="/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28",
        checkpoints=[c]
    )

# for c in checkpoints:
#     plan_dict = eval_helper.run_eval_plan(
#         eval_base_sbatch="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
#         eval_plan=f"stage2_MH19_336px_128nodes_exp_{c}",
#         eval_config_dir="/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval",
#         checkpoint_dir="/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp",
#         checkpoints=[c],
#         rerun_if_exists=False
#     )

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh stage2_MH19_336px_128nodes_exp28_1200 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28 mmmu 1200
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh stage2_MH19_336px_128nodes_exp28_1200 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval/eval_docvqa.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28 docvqa 1200
sbatch --parsable --job-name=eval_mathvista /fsx_0/user/tranx/experiments/eval/sbash_eval.sh stage2_MH19_336px_128nodes_exp28_1200 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval/eval_mathvista.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_128nodes_exp28 mathvista 1200
sbatch --parsable --job-name=eval_ai2d /fsx_0/user/tranx/experiments/eval/sbash_eval.sh stage2_MH19_336px_128nodes_exp28_1200 /fsx_0/use

In [17]:
df = pd.DataFrame()
checkpoints = range(200, 1600, 200)
for c in checkpoints:
    job_dict_file = f"eval/logs/job_dict_stage2_MH19_336px_128nodes_exp_{c}.json"
    with open(job_dict_file, 'r') as f:
        job_dict = json.load(f)

    print(c, job_dict_file)
    df_c = eval_helper.get_eval_scores(job_dict)

    df = pd.concat([df, df_c])

df

200 eval/logs/job_dict_stage2_MH19_336px_128nodes_exp_200.json
Got result for mmmu - 200: [('mmmu/accounting', 0.4), ('mmmu/agriculture', 0.6333333333333333), ('mmmu/architecture_and_engineering', 0.23333333333333334), ('mmmu/art', 0.6666666666666666), ('mmmu/art_theory', 0.9333333333333333), ('mmmu/basic_medical_science', 0.7), ('mmmu/biology', 0.43333333333333335), ('mmmu/chemistry', 0.4), ('mmmu/clinical_medicine', 0.6), ('mmmu/computer_science', 0.6), ('mmmu/design', 0.7666666666666667), ('mmmu/diagnostics_and_laboratory_medicine', 0.36666666666666664), ('mmmu/economics', 0.6666666666666666), ('mmmu/electronics', 0.26666666666666666), ('mmmu/energy_and_power', 0.5333333333333333), ('mmmu/finance', 0.5333333333333333), ('mmmu/geography', 0.5333333333333333), ('mmmu/history', 0.7333333333333333), ('mmmu/literature', 0.8), ('mmmu/manage', 0.5666666666666667), ('mmmu/marketing', 0.43333333333333335), ('mmmu/materials', 0.43333333333333335), ('mmmu/math', 0.3333333333333333), ('mmmu/mec

,mmmu_v2,mmmu_v1,docvqa,mathvista,ai2d,chartqa,vqa,textvqa,infographics_w_ocr,infographics,mmbench
200,0.5556,0.5444,0.649,0.391,0.7937,0.4937,0.7255,0.68126,0.641,0.5379,0.7381
400,0.57,0.5656,0.6534,0.399,0.7895,0.4979,0.7258,0.68664,0.6334,0.5391,0.7353
600,0.5567,0.5433,0.6576,0.379,0.7947,0.4977,0.7262,0.68338,0.6451,0.5388,0.737
800,0.5611,0.5489,0.6599,0.395,0.7927,0.5077,0.7258,0.6852,0.6445,0.5329,0.7383
1000,0.5689,0.5578,0.6627,0.383,0.7924,0.5017,0.7265,0.68932,0.6428,0.5437,0.7367
1200,0.5656,0.5489,0.6629,0.392,0.7966,0.5069,0.7272,0.69168,0.6413,0.5518,0.7438
1400,0.5633,0.5578,0.6576,0.39,0.8015,0.5053,0.7249,0.68468,0.637,0.5447,0.7389


In [22]:
df.to_csv('stage2_eval_3200_4600.csv')

## 504px exp 29

In [12]:
!ls /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp29

checkpoint-1000  checkpoint-200  checkpoint-600  runs
checkpoint-1200  checkpoint-400  checkpoint-800


In [14]:
checkpoints = [200, 400, 600, 800, 1000, 1200]
for c in checkpoints:
    plan_dict = eval_helper.run_eval_plan(
        eval_base_sbatch=EVAL_SBATCH,
        eval_plan=f"stage2_MH19_504px_64nodes_exp29_{c}",
        eval_config_dir=EVAL_CONFIG_DIR,
        checkpoint_dir="/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp29",
        checkpoints=[c]
    )

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh stage2_MH19_504px_64nodes_exp29_200 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp29 mmmu 200
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh stage2_MH19_504px_64nodes_exp29_200 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval/eval_docvqa.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp29 docvqa 200
sbatch --parsable --job-name=eval_mathvista /fsx_0/user/tranx/experiments/eval/sbash_eval.sh stage2_MH19_504px_64nodes_exp29_200 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval/eval_mathvista.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_504px_64nodes_exp29 mathvista 200
sbatch --parsable --job-name=eval_ai2d /fsx_0/user/tranx/experiments/eval/sbash_eval.sh stage2_MH19_504px_64nodes_exp29_200 /fsx_0/user/tranx/eval/l

## 336px i18n

In [15]:
!ls /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_64nodes_i18n

checkpoint-5000  checkpoint-5200  runs


In [16]:
checkpoints = [5000, 5200]
for c in checkpoints:
    plan_dict = eval_helper.run_eval_plan(
        eval_base_sbatch=EVAL_SBATCH,
        eval_plan=f"stage2_MH19_336px_64nodes_i18n_{c}",
        eval_config_dir=EVAL_CONFIG_DIR,
        checkpoint_dir="/fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_64nodes_i18n",
        checkpoints=[c]
    )

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh stage2_MH19_336px_64nodes_i18n_5000 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_64nodes_i18n mmmu 5000
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh stage2_MH19_336px_64nodes_i18n_5000 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval/eval_docvqa.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_64nodes_i18n docvqa 5000
sbatch --parsable --job-name=eval_mathvista /fsx_0/user/tranx/experiments/eval/sbash_eval.sh stage2_MH19_336px_64nodes_i18n_5000 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval/eval_mathvista.json /fsx_0/checkpoints/tranx/MM9-Stage2-70B/MH19_336px_64nodes_i18n mathvista 5000
sbatch --parsable --job-name=eval_ai2d /fsx_0/user/tranx/experiments/eval/sbash_eval.sh stage2_MH19_336px_64nodes_i18n_5000 /fsx_0/user/tranx/eval/l